In [1]:
import pandas as pd 
import numpy as np

In [37]:
basefile = pd.read_csv("data/2020_Claims_SingleSource_v1.4.csv",dtype=str)
newfile = pd.read_csv("data/2021_Claims_SingleSource_v1.3.csv",dtype=str)

basefile.columns = basefile.columns.str.replace(' ', '_')
newfile.columns = newfile.columns.str.replace(' ', '_')
# Version to capture 
basefile["VERSION"] = "BASE"
newfile["VERSION"] = "NEW"
# Const to capture the whole file
basefile["CONST"]="CONST"
newfile["CONST"]="CONST"

### Count comparison

In [3]:
basefile.count()

Measure_ID           19434
DATA_ELEMENT_NAME    19434
CODING_SYSTEM        19434
CODE                 19434
MODIFIER               883
PLACE_OF_SERVICE       724
AGE                  19434
GENDER               19434
VERSION              19434
dtype: int64

In [4]:
newfile.count()

Measure_ID           21380
DATA_ELEMENT_NAME    21380
CODING_SYSTEM        21380
CODE                 21380
MODIFIER               370
PLACE_OF_SERVICE       355
AGE                  21380
GENDER               21380
VERSION              21380
dtype: int64

### Joining the two files 

In [11]:
key=newfile.columns.values.tolist()

In [12]:
key.remove("VERSION")

In [13]:
joint=newfile.merge(basefile,on=key,how="outer",suffixes=["_new","_base"])

In [117]:
def getJoinMeta(row):
    val = "NONE"
    if (row["VERSION_new"] == "NEW") & (row ["VERSION_base"] == "BASE"):
        val = "COMMON"
    elif row["VERSION_new"] == "NEW":
        val = "NEW"
    else: 
        val = "BASE" 
    return val

In [123]:
joint["VERSION"]= joint.apply(getJoinMeta,axis=1)
joint=joint.drop(["VERSION_new","VERSION_base"],axis=1)

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,CONST,VERSION
0,1,ENCOUNTER_CODE,C4,99202,NaN,NaN,18 - 75,"M, F",CONST,COMMON
1,1,ENCOUNTER_CODE,C4,99203,NaN,NaN,18 - 75,"M, F",CONST,COMMON
2,1,ENCOUNTER_CODE,C4,99204,NaN,NaN,18 - 75,"M, F",CONST,COMMON
3,1,ENCOUNTER_CODE,C4,99205,NaN,NaN,18 - 75,"M, F",CONST,COMMON
4,1,ENCOUNTER_CODE,C4,99212,NaN,NaN,18 - 75,"M, F",CONST,COMMON
...,...,...,...,...,...,...,...,...,...,...
25968,437,G_CODE_PD,HCPCS,G9641,NaN,NaN,≥0,"M, F",CONST,BASE
25969,437,G_CODE_PN,HCPCS,G9641,NaN,NaN,≥0,"M, F",CONST,BASE
25970,437,G_CODE_PD_Exl,HCPCS,G9640,NaN,NaN,≥0,"M, F",CONST,BASE
25971,437,G_CODE_PD,HCPCS,G9639,NaN,NaN,≥0,"M, F",CONST,BASE


In [124]:
joint[joint.VERSION=="NONE"]

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,VERSION_new,CONST,VERSION_base,VERSION


In [119]:
joint

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,VERSION_new,CONST,VERSION_base,VERSION
0,1,ENCOUNTER_CODE,C4,99202,NaN,NaN,18 - 75,"M, F",NEW,CONST,BASE,COMMON
1,1,ENCOUNTER_CODE,C4,99203,NaN,NaN,18 - 75,"M, F",NEW,CONST,BASE,COMMON
2,1,ENCOUNTER_CODE,C4,99204,NaN,NaN,18 - 75,"M, F",NEW,CONST,BASE,COMMON
3,1,ENCOUNTER_CODE,C4,99205,NaN,NaN,18 - 75,"M, F",NEW,CONST,BASE,COMMON
4,1,ENCOUNTER_CODE,C4,99212,NaN,NaN,18 - 75,"M, F",NEW,CONST,BASE,COMMON
...,...,...,...,...,...,...,...,...,...,...,...,...
25968,437,G_CODE_PD,HCPCS,G9641,NaN,NaN,≥0,"M, F",NaN,CONST,BASE,BASE
25969,437,G_CODE_PN,HCPCS,G9641,NaN,NaN,≥0,"M, F",NaN,CONST,BASE,BASE
25970,437,G_CODE_PD_Exl,HCPCS,G9640,NaN,NaN,≥0,"M, F",NaN,CONST,BASE,BASE
25971,437,G_CODE_PD,HCPCS,G9639,NaN,NaN,≥0,"M, F",NaN,CONST,BASE,BASE


In [126]:
onlybase=joint[joint.VERSION=="BASE"]
len(onlybase)

4560

In [127]:
onlynew=joint[joint.VERSION=="NEW"]
len(onlynew)

6513

In [128]:
intersection=joint[joint.VERSION=="COMMON"]

In [129]:
len(intersection)

14900

# New against Base comparison

## Defining functions

In [131]:
def columnToSet(df,column:str):
    return set(df[column].unique().tolist())

In [132]:
def analyze_difference(joint,partid,partcolumn,subcol):
    subset=joint[joint[partcolumn]==partid]
    onlybase=subset[subset.VERSION_new.isna()]
    onlynew=subset[subset.VERSION_base.isna()]
    
    onlynewids=columnToSet(onlynew,subcol)
    onlybaseids=columnToSet(onlybase,subcol)
    baseids=columnToSet(onlybase,subcol)
    newids=columnToSet(onlynew,subcol)
    
    added=onlynewids-baseids
    removed=onlybaseids-newids
    changed=onlynewids|onlybaseids-added-removed
    
    print("Added")
    print(added)
    print("Removed")
    print(removed)
    print("Changed")
    print(changed)
    
    return (subset,added|removed|changed) 
    

In [136]:
(df,changed)=analyze_difference(joint,"93","Measure_ID","CODE")

Added
set()
Removed
{'99201'}
Changed
{'99308', '99204', '99334', '99325', '99348', '99336', '99309', '99214', '99202', '99343', '4131F', '99284', '99205', '99213', '99212', '99305', '99341', '99281', '99347', '4132F', '99215', '99335', '99350', '99328', '99285', '99310', '99326', '99306', '99307', '99283', '99324', '99203', '99304', '99282', '99342', '99344', '99349', '99345', '99327'}


### Iterative process

In [137]:
(df,changed)=analyze_difference(joint,"CONST","CONST","Measure_ID")

Added
{'436'}
Removed
{'14', '416', '21', '12', '146', '435', '52', '317', '48', '419', '422', '437', '268'}
Changed
{'326', '418.01', '226.02', '155', '195', '261', '436', '50', '110', '112', '226', '134', '405', '254', '117', '147', '236', '24', '249', '128', '395', '1', '182', '93', '141', '154', '320', '425', '47', '24.01', '145', '39', '113', '225', '250', '406', '76', '130', '226.01', '111', '181', '23', '418'}


In [138]:
(dfL1,changed)=analyze_difference(df,"93","Measure_ID","DATA_ELEMENT_NAME")

Added
set()
Removed
set()
Changed
{'CPT_II_PD_Exe', 'CPT_II_PN', 'CPT_II_PN_X', 'ENCOUNTER_CODE', 'CPT_II_PD'}


In [139]:
(dfL2,changed)=analyze_difference(dfL1,"CPT_II_PD_Exe","DATA_ELEMENT_NAME","CODE")

Added
set()
Removed
set()
Changed
{'4131F'}


In [140]:
dfL2[dfL2.CODE=="4131F"]

,Measure_ID,DATA_ELEMENT_NAME,CODING_SYSTEM,CODE,MODIFIER,PLACE_OF_SERVICE,AGE,GENDER,VERSION_new,CONST,VERSION_base,VERSION
9983,93,CPT_II_PD_Exe,CPT_II,4131F,1P,NaN,≥2,"M, F",NEW,CONST,NaN,NEW
23101,93,CPT_II_PD_Exe,CPT_II,4131F,"1P, ≠ 2P, 3P, 8P",NaN,≥2,"M, F",NaN,CONST,BASE,BASE
